<a href="https://colab.research.google.com/github/adamggibbs/marine-carbonate-system-ml-prediction/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# IMPORTS

import os
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# data dirs
data_dir = '/content/drive/MyDrive/Adam Gibbs/data/'
training_dir=data_dir + 'training/'

In [ ]:
# load in txt data

ship_input = np.loadtxt(data_dir+'ship_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
ship_output = np.loadtxt(data_dir+'ship_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

glider_input = np.loadtxt(data_dir+'glider_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
glider_output = np.loadtxt(data_dir+'glider_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

wcoa_input = np.loadtxt(data_dir+'wcoa_tpso_input.csv', 
                        delimiter=",", 
                        dtype='str')
wcoa_output = np.loadtxt(data_dir+'wcoa_ph_output.csv', 
                        delimiter=",", 
                        dtype='str')

display(ship_input)
display(ship_output)

display(glider_input)
display(glider_output)

display(wcoa_input)
display(wcoa_output)


array([['20210508', '36.762', '-122.038', ..., '6.4717', '34.2377',
        '29.92113696'],
       ['20210508', '36.762', '-122.038', ..., '8.2529', '34.0901',
        '78.45743266'],
       ['20210508', '36.762', '-122.038', ..., '8.5422', '34.0623',
        '88.60173573'],
       ...,
       ['20190516', '36.6977', '-122.263', ..., '6.1254', '34.2424',
        '20.9'],
       ['20190516', '36.6977', '-122.263', ..., '6.6709', '34.1909',
        '33.8'],
       ['20190516', '36.6977', '-122.263', ..., '7.8394', '34.0777',
        '65.2']], dtype='<U18')

array(['7.625306433977267', '7.704910906780151', '7.72242414848945',
       '7.7378456694608655', '7.91654177891317', '7.9343081806234155',
       '7.944870665731484', '7.951711333134443', '7.953563494797294',
       '7.64976663381957', '7.641461973021745', '7.62920294568181',
       '7.614088441318122', '7.636916227529788', '7.752333017149903',
       '7.768713740890694', '7.781676986709455', '7.963560187892795',
       '8.012282014958181', '8.027690424260669', '7.562688340995304',
       '7.5644187792281095', '7.563959288074471', '7.561191415358475',
       '7.5603316753557745', '7.561689934326952', '7.562500095898405',
       '7.5635597612003584', '7.570525170997582', '7.601214034267703',
       '8.157730077954078', '7.565346140111108', '7.557986789506264',
       '7.55823534597981', '7.559913420985177', '7.563858154053043',
       '7.565109074819537', '7.568269938813018', '7.58120201065702',
       '7.602981133625982', '7.622608642250877', '7.615185583344253',
       '7.61666909309

array([['20190508', '36.736', '-122.011', ..., '6.017',
        '34.233000000000004', '19.45'],
       ['20190508', '36.736', '-122.011', ..., '6.026', '34.232',
        '19.42'],
       ['20190508', '36.736', '-122.011', ..., '6.035', '34.231',
        '19.42'],
       ...,
       ['20200318', '36.794', '-121.828', ..., '13.244000000000002',
        '33.525999999999996', '257.75'],
       ['20200318', '36.794', '-121.828', ..., '13.235999999999999',
        '33.528', '258.08'],
       ['20200318', '36.794', '-121.828', ..., '13.245999999999999',
        '33.527', '258.19']], dtype='<U18')

array(['7.5485', '7.5486', '7.5484', ..., '8.0397', '8.0413', '8.0419'],
      dtype='<U18')

array([['20160518', '36.81', '-121.818', '15.0', '13.34', '33.143',
        '272.8'],
       ['20160518', '36.81', '-121.818', '5.1', '13.96',
        '33.138000000000005', '273.9'],
       ['20160518', '36.81', '-121.818', '3.4', '14.0', '33.139',
        '274.0'],
       ['20160518', '36.795', '-121.848', '251.2', '7.89', '34.05',
        '67.8'],
       ['20160518', '36.795', '-121.848', '151.0', '8.62', '33.954',
        '92.6'],
       ['20160518', '36.795', '-121.848', '100.5', '8.87', '33.908',
        '104.4'],
       ['20160518', '36.795', '-121.848', '40.8', '10.09', '33.617',
        '165.0'],
       ['20160518', '36.795', '-121.848', '30.4', '10.17', '33.548',
        '177.7'],
       ['20160518', '36.795', '-121.848', '20.0', '10.53', '33.399',
        '190.5'],
       ['20160518', '36.795', '-121.848', '10.5', '12.65', '33.321',
        '238.8'],
       ['20160518', '36.784', '-121.87200000000001', '100.2', '8.67',
        '33.959', '91.2'],
       ['20160518', '36.784', 

array(['8.034862400919026', '8.040221578006799', '8.042729240792925',
       '7.614367250629397', '7.649846263031173', '7.666234175745569',
       '7.759884604571789', '7.791583909488228', '7.829769056186604',
       '7.9571426289466585', '7.654406486127567', '7.731111957408882',
       '7.792791213405503', '7.844613316074569', '7.934089886224436',
       '7.728975840249242', '7.813043343673816', '7.654713691902718',
       '7.734461970398188', '7.783192154238925', '7.874654237803034',
       '8.020945551070671', '7.547830986271524', '7.587966406959314',
       '7.661293458326776', '7.704925720936962', '8.028133557088248',
       '7.548931548136637', '7.559850661425167', '7.627537898323465',
       '7.689201872403516', '7.7141705029480026', '7.825143189743733',
       '7.889543331973465', '7.625734472182768', '7.552612173292936',
       '7.640436648343181', '7.717278360710171', '7.82518714006605',
       '7.914619909617428', '7.994079254751035', '7.631890490116242',
       '7.538796826

In [ ]:
# CONCATENATE INPUTS
def concatenate_prep_data(input_arrays, output_arrays, 
                          save_csv=False, save_txt=False):

  inputs = np.concatenate(input_arrays)
  canyonb_inputs = np.concatenate(input_arrays)
  outputs = np.concatenate(output_arrays)
  canyonb_outputs = np.concatenate(output_arrays)

  # TRANSFORM DATE AND PRESSURE INPUTS

  # method to help transform date
  def date_to_nth_day(the_date):
    date = pd.to_datetime(the_date)
    new_year_day = pd.Timestamp(year=date.year, month=1, day=1)
    day_of_the_year = (date - new_year_day).days + 1
    return day_of_the_year

  # loop through inputs and perform transformations
  for input in inputs:
    # adjust date
    date = input[0]
    frac_year = date_to_nth_day(date) / 365
    input[0] = int(date[0:4]) + frac_year 
    # # adjust pressure
    # pres = float(input[3])
    # input[3] = ( pres/20000 ) + ( 1 / ( 1 + math.exp(-1*pres / 300) )**3 ) 

  # loop through inputs and perform transformations
  for input in canyonb_inputs:
    # adjust date
    date = input[0]
    date = date[0:4]+'-'+date[4:6]+'-'+date[6:8]
    input[0] = date
    
  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      np.savetxt(data_dir + 'compiled_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'compiled_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      np.savetxt(data_dir + 'compiled_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'compiled_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)
      
  # if desired save arrays as .txt files
  if save_txt or save_csv: 
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      np.savetxt(data_dir + 'canyonb_compiled_tpso_input.txt', inputs, 
                 fmt='%s', header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'canyonb_compiled_ph_output.txt', outputs, 
                 fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      np.savetxt(data_dir + 'canyonb_compiled_tpso_input.csv', inputs, 
                 fmt='%s', delimiter=",", header=input_header)
      output_header = 'TOT_PH'
      np.savetxt(data_dir + 'canyonb_compiled_ph_output.csv', outputs, 
                 fmt='%s', delimiter=",", header=output_header)
      
  return inputs, outputs, canyonb_inputs, canyonb_outputs

# GET INPUT OUTPUT ARRAYS

inputs, outputs, canyonb_inputs, canyonb_outputs = concatenate_prep_data(
                                        (ship_input, glider_input, wcoa_input),
                                        (ship_output, glider_output, wcoa_output),
                                        save_csv=True)

display(inputs[0])
display(outputs[0])

array(['2021.3506849315067', '36.762', '-122.038', '504.8', '6.4717',
       '34.2377', '29.92113696'], dtype='<U19')

'7.625306433977267'

In [ ]:
display(inputs.shape)
display(canyonb_inputs.shape)

(642860, 7)

(642860, 7)

In [ ]:
def split_input_output(inputs, outputs, test_size=0.2, val_size=0.25,
                       seed=101, save_txt=False, save_csv=False):
  train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(
      inputs, outputs, test_size=test_size, random_state=seed
      )

  train_inputs, val_inputs, train_outputs, val_outputs = train_test_split(
      train_inputs, train_outputs, test_size=val_size, random_state=seed
      )
  
  # split CANYON-B
  canyonb_train_inputs, canyonb_test_inputs, canyonb_train_outputs, canyonb_test_outputs = train_test_split(
      canyonb_inputs, canyonb_outputs, 
      test_size=test_size, random_state=seed
      )

  canyonb_train_inputs, canyonb_val_inputs, canyonb_train_outputs, canyonb_val_outputs = train_test_split(
      canyonb_train_inputs, canyonb_train_outputs, 
      test_size=val_size, random_state=seed
      )

  if save_txt or save_csv:
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'training_tpso_input.txt', train_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'training_ph_output.txt', train_outputs, 
                  fmt='%s', header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'validation_tpso_input.txt', val_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'validation_ph_output.txt', val_outputs, 
                  fmt='%s', header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'testing_tpso_input.txt', test_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'testing_ph_output.txt', test_outputs, 
                  fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'training_tpso_input.csv', train_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'training_ph_output.csv', train_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'validation_tpso_input.csv', val_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'validation_ph_output.csv', val_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'testing_tpso_input.csv', test_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'testing_ph_output.csv', test_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      
  if save_txt or save_csv:
    if save_txt:
      input_header = 'DATE LATITUDE LONGITUDE CTDPRS CTDTMP CTDSAL OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'canyonb_training_tpso_input.txt', canyonb_train_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_training_ph_output.txt', canyonb_train_outputs, 
                  fmt='%s', header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'canyonb_validation_tpso_input.txt', canyonb_val_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_validation_ph_output.txt', canyonb_val_outputs, 
                  fmt='%s', header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'canyonb_testing_tpso_input.txt', canyonb_test_inputs, 
                  fmt='%s', header=input_header)
      np.savetxt(training_dir + 'canyonb_testing_ph_output.txt', canyonb_test_outputs, 
                  fmt='%s', header=output_header)
    else:
      input_header = 'DATE, LATITUDE, LONGITUDE, CTDPRS, CTDTMP, CTDSAL, OXYGEN'
      output_header = 'TOT_PH'
      # save training sets
      np.savetxt(training_dir + 'canyonb_training_tpso_input.csv', canyonb_train_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_training_ph_output.csv', canyonb_train_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save validation sets
      np.savetxt(training_dir + 'canyonb_validation_tpso_input.csv', canyonb_val_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_validation_ph_output.csv', canyonb_val_outputs, 
                  fmt='%s', delimiter=",", header=output_header)
      # save testing sets
      np.savetxt(training_dir + 'canyonb_testing_tpso_input.csv', canyonb_test_inputs, 
                  fmt='%s', delimiter=",", header=input_header)
      np.savetxt(training_dir + 'canyonb_testing_ph_output.csv', canyonb_test_outputs, 
                  fmt='%s', delimiter=",", header=output_header)

  return train_inputs, train_outputs, val_inputs, val_outputs, test_inputs, test_outputs


train_inputs, train_outputs, val_inputs, val_outputs, test_inputs, test_outputs = split_input_output(
    inputs, outputs, test_size=0.2, val_size=0.25, save_csv=True
)

display(train_inputs.shape)
display(train_outputs.shape)

display(val_inputs.shape)
display(val_outputs.shape)

display(test_inputs.shape)
display(test_outputs.shape)


(385716, 7)

(385716,)

(128572, 7)

(128572,)

(128572, 7)

(128572,)

In [ ]:
# (385716, 7)
# (385716,)
# (128572, 7)
# (128572,)
# (128572, 7)
# (128572,)